In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from scipy.optimize import minimize
import os

In [2]:
df_1= pd.read_excel(r'C:\Users\Acer\Documents\python\proyek_antam\Merge Table to_RK and to_EF part 2.xlsx', sheet_name='shifting_7-7_16', skiprows=0)
# df_1 = df_1.rename(columns={
#     'Unnamed: 0': 'date',
#     'Unnamed: 1': 'time',
#     # tambahkan kolom lain sesuai kebutuhan
# })
df_1

,date,time,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,...,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,2021-01-01,00:00:00,2.090,12.680,37.850,0.420,20.130,1.25,6.060000,1.880000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-01,01:00:00,2.090,12.680,37.850,0.420,20.130,1.25,6.060000,1.880000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-01,02:00:00,2.090,12.680,37.850,0.420,20.130,1.25,6.060000,1.880000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-01,03:00:00,2.090,12.680,37.850,0.420,20.130,1.25,6.060000,1.880000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-01,04:00:00,2.090,12.680,37.850,0.420,20.130,1.25,6.060000,1.880000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,2021-01-31,19:00:00,2.115,14.555,37.005,0.575,19.845,2.77,6.912904,1.864503,...,17.99,1.483,1.757,76.737,0.4162,0.055,5.651667,612.0,-3.3,0.55
603,2021-01-31,20:00:00,2.115,14.555,37.005,0.575,19.845,2.77,6.912904,1.864503,...,17.99,1.483,1.757,76.737,0.4162,0.055,5.651667,615.0,-5.0,0.55
604,2021-01-31,21:00:00,2.115,14.555,37.005,0.575,19.845,2.77,6.912904,1.864503,...,17.99,1.483,1.757,76.737,0.4162,0.055,5.651667,620.0,-2.4,0.54
605,2021-01-31,22:00:00,2.115,14.555,37.005,0.575,19.845,2.77,6.912904,1.864503,...,17.99,1.483,1.757,76.737,0.4162,0.055,5.651667,617.0,-3.2,0.54


In [3]:
df = df_1.drop(columns=['date', 'time']).copy()
df.columns
df.head()

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,2.09,12.68,37.85,0.42,20.13,1.25,6.06,1.88,0.54,11.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.09,12.68,37.85,0.42,20.13,1.25,6.06,1.88,0.54,11.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.09,12.68,37.85,0.42,20.13,1.25,6.06,1.88,0.54,11.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.09,12.68,37.85,0.42,20.13,1.25,6.06,1.88,0.54,11.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.09,12.68,37.85,0.42,20.13,1.25,6.06,1.88,0.54,11.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df.drop(index=range(0, 16)).reset_index(drop=True)
df = df.iloc[:-7].reset_index(drop=True)
input_cols = df.loc[:, 'ni_in':'t_tic172'].columns
output_cols = df.loc[:, 'furnace_temp':'loi_kalsin'].columns

In [5]:
df[input_cols.union(output_cols)] = df[input_cols.union(output_cols)].interpolate(method='linear', axis=0)
print(df[input_cols.union(output_cols)].isnull().sum())

a_f_ratio           0
al2o3_in            0
bc                  0
c_met               0
cao_in              0
charge_kiln         0
current             0
fc_coal             0
fe_in               0
fe_met              0
fe_ni               0
fe_slag             0
furnace_temp        0
gcv_coal            0
kg_tco              0
load                0
loi_in              0
loi_kalsin          0
mc_kilnfeed         0
mgo_in              0
ni_in               0
ni_met              0
ni_slag             0
pic_161             0
pry_p               0
pry_v               0
reductor_consume    0
reductor_ratio      0
rpm                 0
s_m                 0
s_met               0
sec_p               0
sec_v               0
si_met              0
sio2_in             0
t_kalsin            0
t_tic162            0
t_tic163            0
t_tic166            0
t_tic172            0
tco                 0
total_coal          0
voltage             0
dtype: int64


In [6]:
# X_merged = pd.concat([normalized_df[input_cols], pca_df], axis=1)
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_norm = pd.DataFrame(scaler_x.fit_transform(df[input_cols]), columns=input_cols)
y_norm = pd.DataFrame(scaler_y.fit_transform(df[output_cols]), columns=output_cols)
# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)
X_train

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,total_coal,a_f_ratio,kg_tco,reductor_ratio,reductor_consume,charge_kiln,t_tic162,t_tic163,t_tic166,t_tic172
77,0.873684,0.566532,0.695225,0.349515,0.414134,0.560510,0.358249,0.478152,0.388446,0.112738,...,0.666770,0.185681,0.048184,0.633803,0.612884,0.771142,0.651123,0.865815,0.274747,0.531250
436,0.757895,0.892137,0.228933,0.582524,0.473498,0.996815,0.709085,0.199270,0.734372,0.000000,...,0.323283,0.220923,0.041159,0.718310,0.865563,0.895877,0.436960,0.706070,0.204040,0.364583
211,0.652632,0.704637,0.655899,0.893204,0.270671,0.925159,0.606827,0.606325,0.310117,0.023426,...,0.626360,0.194009,0.041898,0.570423,0.751461,0.903422,0.594128,0.843450,0.432323,0.510417
192,0.621053,0.584677,0.685393,0.388350,0.402827,0.598726,0.524448,0.493338,0.388446,0.071742,...,0.545539,0.256850,0.044298,0.637324,0.884681,1.000000,0.656304,0.699681,0.345455,0.562500
450,0.494737,0.497984,0.470506,0.524272,0.725795,0.821656,0.510112,0.116077,0.833450,0.135432,...,0.646565,0.144896,0.043059,0.806338,0.896736,0.850323,0.578584,0.779553,0.373737,0.781250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0.736842,0.204637,0.811096,0.223301,1.000000,0.447452,0.108579,0.076960,0.878788,0.381406,...,0.484924,0.304050,0.048511,0.647887,0.612884,0.660744,0.787565,0.627263,0.356902,0.715278
106,0.852632,0.419355,0.678371,0.155340,0.658657,0.496815,0.247451,0.265543,0.655052,0.434846,...,0.545539,0.247601,0.049039,0.697183,0.676571,0.694407,0.658031,0.837061,0.383838,0.770833
270,0.557895,0.618952,0.766854,0.660194,0.166784,0.914013,0.590927,0.781879,0.175160,0.111274,...,0.663876,0.180807,0.041659,0.714789,0.849001,0.256860,0.571675,0.690096,0.309091,0.604167
435,0.757895,0.892137,0.228933,0.582524,0.473498,0.996815,0.709085,0.199270,0.734372,0.000000,...,0.495026,0.198373,0.043042,0.718310,0.865563,0.895877,0.069085,0.057508,0.115152,0.177083


In [7]:
# Initialize Ridge Regression model
ridge_model = Ridge(alpha=0.1, solver='svd')  # Regularization strength

# Train the model
ridge_model.fit(X_train.to_numpy(), y_train)

# Predict on test set
y_pred_rr = ridge_model.predict(X_test.to_numpy())
y_pred_rr = scaler_y.inverse_transform(y_pred_rr)
y_pred_rr_df = pd.DataFrame(y_pred_rr, columns=output_cols)
X_test_inverted = scaler_x.inverse_transform(X_test)
X_test_inverted = pd.DataFrame(X_test_inverted, columns=input_cols)
inverted_pred_rr_df = pd.concat([X_test_inverted.reset_index(drop=True), y_pred_rr_df.reset_index(drop=True)], axis=1)
inverted_pred_rr_df

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,2.055,13.265,39.840,0.460,23.195,1.800,6.451669,1.718814,0.590599,13.195,...,21.434619,1.032012,1.138098,74.659137,0.522043,0.061797,9.853076,642.742763,-3.596433,0.200082
1,2.070,15.505,37.615,0.510,20.935,2.930,7.491014,1.800052,0.566749,10.155,...,21.466702,1.185195,1.161662,74.394385,0.579230,0.066179,8.777381,623.463701,-3.224005,0.238611
2,2.070,12.310,42.130,0.445,22.910,1.505,5.949930,1.838900,0.551500,11.580,...,21.890556,1.258247,1.222987,73.836141,0.525850,0.067656,8.079174,604.315147,-3.490952,0.588601
3,2.055,15.165,37.190,0.560,21.310,0.000,7.378908,1.746586,0.583696,10.305,...,21.373712,1.134551,1.070287,74.721941,0.496186,0.063651,9.846618,614.190810,-3.309884,-0.082742
4,1.960,15.410,35.165,0.570,19.755,3.005,7.861458,1.780050,0.573375,14.105,...,21.877754,1.062699,1.000229,74.388752,0.521484,0.063069,10.250483,618.134206,-3.337657,0.038351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,2.070,15.200,36.470,0.550,21.290,2.980,7.350000,1.710000,0.590000,10.320,...,21.581631,1.194465,1.217031,74.289400,0.478729,0.058827,8.836824,619.819149,-3.323341,0.108005
113,1.995,14.075,38.155,0.610,20.610,2.620,7.055126,1.851237,0.551703,10.870,...,19.880029,1.290462,1.470838,75.576145,0.441067,0.051607,4.881870,655.927740,-3.344603,0.045574
114,2.055,12.525,40.580,0.450,22.610,1.870,6.100760,1.795170,0.565000,11.925,...,21.891389,1.257396,1.241540,73.853903,0.500607,0.068438,7.993305,606.324216,-3.149074,0.415379
115,2.030,14.775,36.970,0.595,21.000,2.985,7.281335,1.761539,0.579661,10.035,...,21.062814,1.028139,0.980713,75.244888,0.530243,0.061193,8.954138,644.120816,-2.653998,0.430996


In [8]:
inverted_y_actual = scaler_y.inverse_transform(y_test)
inverted_y_actual = pd.DataFrame(inverted_y_actual, columns=output_cols)
inverted_y_pred_rr = y_pred_rr_df[output_cols]
inverted_y_pred_rr

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,1378.638769,21.434619,1.032012,1.138098,74.659137,0.522043,0.061797,9.853076,642.742763,-3.596433,0.200082
1,1373.096868,21.466702,1.185195,1.161662,74.394385,0.579230,0.066179,8.777381,623.463701,-3.224005,0.238611
2,1377.305611,21.890556,1.258247,1.222987,73.836141,0.525850,0.067656,8.079174,604.315147,-3.490952,0.588601
3,1383.747788,21.373712,1.134551,1.070287,74.721941,0.496186,0.063651,9.846618,614.190810,-3.309884,-0.082742
4,1379.140036,21.877754,1.062699,1.000229,74.388752,0.521484,0.063069,10.250483,618.134206,-3.337657,0.038351
...,...,...,...,...,...,...,...,...,...,...,...
112,1384.442789,21.581631,1.194465,1.217031,74.289400,0.478729,0.058827,8.836824,619.819149,-3.323341,0.108005
113,1399.534726,19.880029,1.290462,1.470838,75.576145,0.441067,0.051607,4.881870,655.927740,-3.344603,0.045574
114,1390.519631,21.891389,1.257396,1.241540,73.853903,0.500607,0.068438,7.993305,606.324216,-3.149074,0.415379
115,1356.460567,21.062814,1.028139,0.980713,75.244888,0.530243,0.061193,8.954138,644.120816,-2.653998,0.430996


In [9]:
inverted_y_actual

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,1377.000000,21.445000,1.030000,1.125000,74.661500,0.528500,0.064000,10.174000,640.000000,-4.000000,0.00
1,1374.500000,21.173333,1.188333,1.158333,74.656833,0.614833,0.066667,9.256667,660.000000,-2.700000,0.68
2,1346.400000,21.084000,1.326000,1.172000,74.524200,0.569800,0.070000,8.080000,611.000000,-3.800000,0.54
3,1386.666667,21.540000,1.130000,1.086667,74.542000,0.508667,0.064000,10.108000,592.000000,-3.100000,0.00
4,1373.000000,21.647500,1.065000,0.972500,74.673000,0.486000,0.062500,10.165000,635.285714,-3.357143,0.37
...,...,...,...,...,...,...,...,...,...,...,...
112,1393.666667,21.778333,1.223333,1.236667,74.087500,0.445833,0.055000,8.595000,638.250000,-3.125000,0.00
113,1394.875000,19.708750,1.256250,1.436250,75.828000,0.439500,0.050000,4.990000,653.000000,-1.700000,0.00
114,1389.000000,21.947500,1.262500,1.225000,73.847250,0.488750,0.065000,7.810000,602.000000,-2.400000,0.48
115,1394.875000,19.708750,1.256250,1.436250,75.828000,0.439500,0.050000,4.990000,612.000000,-2.000000,0.65


In [10]:
models = {
    'RR': inverted_y_pred_rr,

}
mae_dict = {'Column': output_cols}
# Iterate through each model and calculate MAE for each column
for model_name, inverted_y_pred in models.items():
    mae_dict[model_name] = []  # Add a column for each model
    for col in output_cols:
        # Extract actual and predicted values
        y_actual_col = inverted_y_actual[col]
        y_pred_col = inverted_y_pred[col]

        # Calculate MAE and store in the dictionary
        mae_col = mean_absolute_error(y_actual_col, y_pred_col)
        mae_dict[model_name].append(mae_col)

# Convert the dictionary to a DataFrame
metrics_df = pd.DataFrame(mae_dict)

# Display the DataFrame
display(metrics_df)

,Column,RR
0,furnace_temp,9.292703
1,ni_met,0.256619
2,c_met,0.032770
3,si_met,0.024445
4,fe_met,0.218259
5,s_met,0.032141
6,ni_slag,0.003389
7,fe_slag,0.268211
8,t_kalsin,17.689237
9,pic_161,0.460935


# TANPA CONSTRAINT

In [11]:
target_value = y_test.iloc[0, :].to_numpy()
print(target_value)

def objective_function(x_scaled):
    # untuk  prediksi 1 fitur output saja dengan i sample saja pada 30 fitur input
    x_scaled=x_scaled.reshape(1,-1)
    y_pred = ridge_model.predict(x_scaled)[0]
    # print(ridge_model.predict(x_scaled))
    return (y_pred[0] - target_value[0]) ** 2

# def objective_function(x_scaled):
#     # untuk  prediksi 1 fitur output saja dengan i sample saja pada 30 fitur input
#     x_scaled=x_scaled.reshape(1,-1)
#     y_pred = ridge_model.predict(x_scaled)[0]
#     # print(ridge_model.predict(x_scaled))
#     return y_pred[0]


[0.5177665  0.77698651 0.         0.19439133 0.37896469 0.50631199
 0.63636364 1.         0.5648855  0.21917808 0.        ]


In [12]:
# untuk  1 sample saja pada 30 fitur input
x0=X_test.iloc[0,:].to_numpy().flatten()


result = minimize(objective_function, x0, method='SLSQP')

# Extract the optimized inputs and predicted output
optimized_x = result.x
print("optimized_x",optimized_x)
optimized_y = ridge_model.predict(optimized_x.reshape(1,-1))[0]

print("optimized_y:",optimized_y)


optimized_x [ 7.14527133e-01  4.64704947e-01  6.80025147e-01  3.28874382e-01
  6.77570303e-01  5.73934316e-01  3.49831219e-01  2.41207323e-01
  6.53734363e-01  5.28439774e-01  7.79032107e-01  7.11556002e-01
 -5.41996938e-05  3.20263337e-01  8.04808412e-01  1.00044479e+00
  2.30322310e-01  6.14303957e-01  8.80449998e-01  9.80489323e-01
  9.38682397e-01  9.83719881e-01  7.26330946e-01  1.43874134e-01
  5.24467094e-02  7.42524893e-01  7.90961779e-01  7.90940086e-01
  5.87533354e-01  7.47778945e-01  4.25800652e-01  7.39207046e-01]
optimized_y: [5.17530321e-01 7.74490502e-01 8.83039379e-04 2.06276006e-01
 3.79027893e-01 4.95376975e-01 5.44964897e-01 9.39928513e-01
 5.70158596e-01 2.74743530e-01 1.99899765e-01]


In [13]:
a=pd.DataFrame(optimized_x).T
a.columns=input_cols
a=pd.DataFrame(scaler_x.inverse_transform(a), columns=input_cols)
print(a.shape)
b=pd.DataFrame(optimized_y).T
b=pd.DataFrame(scaler_y.inverse_transform(b),columns=output_cols)
print(b.shape)

(1, 32)
(1, 11)


In [14]:
a

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,total_coal,a_f_ratio,kg_tco,reductor_ratio,reductor_consume,charge_kiln,t_tic162,t_tic163,t_tic166,t_tic172
0,2.0544,13.274937,39.851779,0.45937,23.18381,1.802154,6.447207,1.713975,0.589753,13.189244,...,5394.779465,7.080657,53.858743,6.438771,144.173781,2243.060224,962.181812,766.05481,582.771323,289.963876


In [15]:
X_test_inverted.head()

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,total_coal,a_f_ratio,kg_tco,reductor_ratio,reductor_consume,charge_kiln,t_tic162,t_tic163,t_tic166,t_tic172
0,2.055,13.265,39.840,0.460,23.195,1.800,6.451669,1.718814,0.590599,13.195,...,5400.0,7.091111,53.328066,6.43,144.21,2242.24,964.000000,765.000000,584.000000,290.000000
1,2.070,15.505,37.615,0.510,20.935,2.930,7.491014,1.800052,0.566749,10.155,...,5000.0,7.420800,49.324258,6.37,134.23,2358.93,946.000000,815.000000,570.000000,276.000000
2,2.070,12.310,42.130,0.445,22.910,1.505,5.949930,1.838900,0.551500,11.580,...,5200.0,7.382308,57.784198,6.13,136.10,2221.20,963.000000,812.000000,503.000000,285.000000
3,2.055,15.165,37.190,0.560,21.310,0.000,7.378908,1.746586,0.583696,10.305,...,4700.0,7.382553,43.418014,6.67,150.41,2254.15,919.000000,756.000000,550.000000,260.000000
4,1.960,15.410,35.165,0.570,19.755,3.005,7.861458,1.780050,0.573375,14.105,...,4100.0,7.925854,47.012957,7.17,161.34,2249.42,906.071429,765.571429,534.785714,267.642857


In [16]:
b

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,1376.986042,21.433901,1.0304,1.134324,74.661711,0.526075,0.061989,9.862589,641.381552,-3.594372,0.207896


In [17]:
y_test_inverted = scaler_y.inverse_transform(y_test)
y_test_inverted = pd.DataFrame(y_test_inverted, columns=output_cols)
y_test_inverted.head()

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,1377.000000,21.445000,1.030000,1.125000,74.661500,0.528500,0.064000,10.174000,640.000000,-4.000000,0.00
1,1374.500000,21.173333,1.188333,1.158333,74.656833,0.614833,0.066667,9.256667,660.000000,-2.700000,0.68
2,1346.400000,21.084000,1.326000,1.172000,74.524200,0.569800,0.070000,8.080000,611.000000,-3.800000,0.54
3,1386.666667,21.540000,1.130000,1.086667,74.542000,0.508667,0.064000,10.108000,592.000000,-3.100000,0.00
4,1373.000000,21.647500,1.065000,0.972500,74.673000,0.486000,0.062500,10.165000,635.285714,-3.357143,0.37


# MENGGUNAKAN CONSTRAINT

In [18]:
min_values = scaler_y.data_min_
max_values = scaler_y.data_max_

print(min_values.shape)
print(max_values.shape)
konstrain=[]
bounds=[ (i,j) for i, j in zip(min_values, max_values)]
print(bounds)

T_furnace=1300
T_furnace_norm= (T_furnace-bounds[0][0])/(bounds[0][1]-bounds[1][0])
print("T_furnace_norm",T_furnace_norm)

ni_met=17
ni_met_norm= (ni_met-bounds[1][0])/(bounds[1][1]-bounds[1][0])
print("Ni_norm",ni_met_norm)

C_met_low=0
C_met_low_norm= (C_met_low-bounds[2][0])/(bounds[2][1]-bounds[2][0])
print("C_met_low_norm",C_met_low_norm)


Si_met_low=1
Si_met_low_norm= (Si_met_low-bounds[3][0])/(bounds[3][1]-bounds[3][0])
print("Si_met_low_norm",Si_met_low_norm)

Si_met_high=2
Si_met_high_norm= (Si_met_high-bounds[3][0])/(bounds[3][1]-bounds[3][0])
print("Si_met_high_norm",Si_met_high_norm)


fe_met_low=0
fe_met_low_norm= (fe_met_low-bounds[4][0])/(bounds[4][1]-bounds[4][0])
print("fe_met_norm",fe_met_low_norm)

s_met_low=0
s_met_low_norm= (s_met_low-bounds[5][0])/(bounds[5][1]-bounds[5][0])
print("s_met_norm",s_met_low_norm)

s_met_high=0.4
s_met_high_norm= (s_met_high-bounds[5][0])/(bounds[5][1]-bounds[5][0])
print("s_met_high_norm",s_met_high_norm)

ni_slag_low=0
ni_slag_low_norm= (ni_slag_low-bounds[6][0])/(bounds[6][1]-bounds[6][0])
print("ni_slag_low_norm",ni_slag_low_norm)

fe_slag_low=0
fe_slag_low_norm= (fe_slag_low-bounds[7][0])/(bounds[7][1]-bounds[7][0])
print("fe_slag_low_norm",fe_slag_low_norm)


T_kalsin_low=600
T_kalsin_low_norm= (T_kalsin_low-bounds[8][0])/(bounds[8][1]-bounds[8][0])
print("T_kalsin_low_norm",T_kalsin_low_norm)

pic_161_low=-1
pic_161_low_norm= (pic_161_low-bounds[9][0])/(bounds[9][1]-bounds[9][0])
print("pic_161_low_norm",pic_161_low_norm)

loi_kalsin_low=0
loi_kalsin_low_norm= (loi_kalsin_low-bounds[10][0])/(bounds[10][1]-bounds[10][0])
print("loi_kalsin_low_norm",loi_kalsin_low_norm)

loi_kalsin_high=1
loi_kalsin_high_norm= (loi_kalsin_high-bounds[10][0])/(bounds[10][1]-bounds[10][0])
print("loi_kalsin_high_norm",loi_kalsin_high_norm)




(11,)
(11,)
[(1346.4, 1405.5), (17.990000000000002, 22.436666666666664), (1.03, 1.4829999999999999), (0.9725, 1.7570000000000001), (73.395, 76.737), (0.4161999999999999, 0.6379999999999999), (0.05, 0.07200000000000001), (4.99, 10.174000000000001), (492.0, 754.0), (-5.6, 1.7), (0.0, 1.04)]
T_furnace_norm -0.03344120042378079
Ni_norm -0.22263868065967085
C_met_low_norm -2.2737306843267118
Si_met_low_norm 0.0350541746335245
Si_met_high_norm 1.3097514340344165
fe_met_norm -21.961400359066435
s_met_norm -1.876465284039675
s_met_high_norm -0.07303877366997241
ni_slag_low_norm -2.2727272727272725
fe_slag_low_norm -0.962577160493827
T_kalsin_low_norm 0.4122137404580153
pic_161_low_norm 0.6301369863013698
loi_kalsin_low_norm 0.0
loi_kalsin_high_norm 0.9615384615384615


In [19]:
x0 = X_test.iloc[0,:].to_numpy().flatten()
print(x0.shape)
min_values_input = scaler_x.data_min_
max_values_input = scaler_x.data_max_

print(min_values_input.shape)
print(max_values_input.shape)
konstrain_input=[ (i,j) for i, j in zip(min_values_input, max_values_input)]
print(konstrain_input)

ni_in_min=0
ni_in_min_norm= (ni_in_min-konstrain_input[0][0])/(konstrain_input[0][1]-konstrain_input[0][0])
print("ni_in_min_norm",ni_in_min_norm)

fe_in_min=0
fe_in_min_norm= (fe_in_min-konstrain_input[1][0])/(konstrain_input[1][1]-konstrain_input[1][0])
print("fe_in_min_norm",fe_in_min_norm)

sio2_in_min=0
sio2_in_min_norm= (sio2_in_min-konstrain_input[2][0])/(konstrain_input[2][1]-konstrain_input[2][0])
print("sio2_in_min_norm",sio2_in_min_norm)

cao_in_min=0
cao_in_min_norm= (cao_in_min-konstrain_input[3][0])/(konstrain_input[3][1]-konstrain_input[3][0])
print("cao_in_min_norm",cao_in_min_norm)

mgo_in_min=0
mgo_in_min_norm= (mgo_in_min-konstrain_input[4][0])/(konstrain_input[4][1]-konstrain_input[4][0])
print("mgo_in_min_norm",mgo_in_min_norm)



al2o3_in_min=0
al2o3_in_min_norm= (al2o3_in_min-konstrain_input[5][0])/(konstrain_input[5][1]-konstrain_input[5][0])
print("al2o3_in_min_norm",al2o3_in_min_norm)

fe_ni_min=0
fe_ni_min_norm= (fe_ni_min-konstrain_input[6][0])/(konstrain_input[6][1]-konstrain_input[6][0])
print("fe_ni_min_norm",fe_ni_min_norm)

s_m_min=0
s_m_min_norm= (s_m_min-konstrain_input[7][0])/(konstrain_input[7][1]-konstrain_input[7][0])
print("s_m_min_norm",s_m_min_norm)

bc_min=0
bc_min_norm= (bc_min-konstrain_input[8][0])/(konstrain_input[8][1]-konstrain_input[8][0])
print("bc_min_norm",bc_min_norm)

loi_in_min=0
loi_in_min_norm= (loi_in_min-konstrain_input[9][0])/(konstrain_input[9][1]-konstrain_input[9][0])
print("loi_in_min_norm",loi_in_min_norm)



######################################
mc_kilnfeed_exact_norm=x0[10]
# mc_kilnfeed_exact_norm= (mc_kilnfeed_exact-konstrain_input[10][0])/(konstrain_input[10][1]-konstrain_input[10][0])
print("mc_kilnfeed_exact_norm",mc_kilnfeed_exact_norm)

fc_coal_exact_norm=x0[11]
#fc_coal_exact_norm= (fc_coal_exact-konstrain_input[11][0])/(konstrain_input[11][1]-konstrain_input[11][0])
print("fc_coal_exact_norm",fc_coal_exact_norm)

gvc_coal_exact_norm=x0[12]
#gvc_coal_exact_norm= (gvc_coal_exact-konstrain_input[12][0])/(konstrain_input[12][1]-konstrain_input[12][0])
print("gvc_coal_exact_norm",gvc_coal_exact_norm)

tco_exact_norm=x0[13]
#tco_exact_norm= (tco_exact-konstrain_input[13][0])/(konstrain_input[13][1]-konstrain_input[13][0])
print("tco_exact_norm",tco_exact_norm)
#########################################

voltage_min=0
voltage_min_norm= (voltage_min-konstrain_input[14][0])/(konstrain_input[14][1]-konstrain_input[14][0])
print("voltage_min_norm",voltage_min_norm)

current_min=0
current_min_norm= (current_min-konstrain_input[15][0])/(konstrain_input[15][1]-konstrain_input[15][0])
print("current_min_norm",current_min_norm)

load_min=0
load_min_norm= (load_min-konstrain_input[16][0])/(konstrain_input[16][1]-konstrain_input[16][0])
print("load_min_norm",load_min_norm)

rpm_min=0
rpm_min_norm= (rpm_min-konstrain_input[17][0])/(konstrain_input[17][1]-konstrain_input[17][0])
print("rpm_min_norm",rpm_min_norm)

pry_p_min=0
pry_p_min_norm= (pry_p_min-konstrain_input[18][0])/(konstrain_input[18][1]-konstrain_input[18][0])
print("pry_p_min_norm",pry_p_min_norm)

sec_p_min=0
sec_p_min_norm= (sec_p_min-konstrain_input[19][0])/(konstrain_input[19][1]-konstrain_input[19][0])
print("sec_p_min_norm",sec_p_min_norm)

pry_v_min=0
pry_v_min_norm= (pry_v_min-konstrain_input[20][0])/(konstrain_input[20][1]-konstrain_input[20][0])
print("pry_v_min_norm",pry_v_min_norm)

sec_v_min=0
sec_v_min_norm= (sec_v_min-konstrain_input[21][0])/(konstrain_input[21][1]-konstrain_input[21][0])
print("sec_v_min_norm",sec_v_min_norm)


total_coal_min=0
total_coal_min_norm= (total_coal_min-konstrain_input[22][0])/(konstrain_input[22][1]-konstrain_input[22][0])
print("total_coal_norm",total_coal_min_norm)

a_f_ratio_min=0
a_f_ratio_min_norm= (a_f_ratio_min-konstrain_input[23][0])/(konstrain_input[23][1]-konstrain_input[23][0])
print("a_f_ratio_min_norm",a_f_ratio_min_norm)

############################################
kg_tco_exact_norm=x0[24]

#kg_tco_exact_norm= (kg_tco_exact-konstrain_input[24][0])/(konstrain_input[24][1]-konstrain_input[24][0])
#invers_norm=kg_tco_exact_norm*(konstrain_input[24][1]-konstrain_input[24][0])+konstrain_input[24][0]
print("kg_tco_exact_norm",kg_tco_exact_norm)
############################################

reductor_ratio_min=0
reductor_ratio_min_norm= (reductor_ratio_min-konstrain_input[25][0])/(konstrain_input[25][1]-konstrain_input[25][0])
print("reductor_ratio_min_norm",reductor_ratio_min_norm)

reductor_consume_min=0
reductor_consume_min_norm= (reductor_consume_min-konstrain_input[26][0])/(konstrain_input[26][1]-konstrain_input[26][0])
print("reductor_consume_min_norm",reductor_consume_min_norm)

############################################
charge_kiln_exact_norm=x0[27]
#charge_kiln_norm= (charge_kiln-konstrain_input[27][0])/(konstrain_input[27][1]-konstrain_input[27][0])
print("charge_kiln_norm",charge_kiln_exact_norm)
############################################


t_tic162_min=556
t_tic162_min_norm= (t_tic162_min-konstrain_input[28][0])/(konstrain_input[28][1]-konstrain_input[28][0])
print("t_tic162_min_norm",t_tic162_min_norm)

t_tic162_max=1201
t_tic162_max_norm= (t_tic162_max-konstrain_input[28][0])/(konstrain_input[28][1]-konstrain_input[28][0])
print("t_tic162_max_norm",t_tic162_max_norm)

t_tic163_min=456
t_tic163_min_norm= (t_tic163_min-konstrain_input[29][0])/(konstrain_input[29][1]-konstrain_input[29][0])
print("t_tic163_min_norm",t_tic163_min_norm)

t_tic163_max=845
t_tic163_max_norm= (t_tic163_max-konstrain_input[29][0])/(konstrain_input[30][1]-konstrain_input[30][0])
print("t_tic163_max_norm",t_tic163_max_norm)

t_tic166_min=372
t_tic166_min_norm= (t_tic166_min-konstrain_input[30][0])/(konstrain_input[30][1]-konstrain_input[30][0])
print("t_tic166_min_norm",t_tic166_min_norm)

t_tic166_max=867
t_tic166_max_norm= (t_tic166_max-konstrain_input[30][0])/(konstrain_input[30][1]-konstrain_input[30][0])
print("t_tic166_max_norm",t_tic166_max_norm)

t_tic172_min=210
t_tic172_min_norm= (t_tic172_min-konstrain_input[31][0])/(konstrain_input[31][1]-konstrain_input[31][0])
print("t_tic172_min_norm",t_tic172_min_norm)

t_tic172_max=315
t_tic172_max_norm= (t_tic172_max-konstrain_input[31][0])/(konstrain_input[31][1]-konstrain_input[31][0])
print("t_tic172_max_norm",t_tic172_max_norm)



(32,)
(32,)
(32,)
[(1.7149999999999999, 2.19), (10.97, 15.93), (35.01, 42.13), (0.29, 0.805), (18.39, 25.465), (0.0, 3.14), (5.5, 8.207610691695573), (1.55514315146416, 2.21363), (0.46715, 0.6546923759479748), (9.58, 16.41), (12.505, 23.325000000000003), (34.0, 49.05), (658.97, 8485.76), (0.0, 448.94000244140625), (734.1, 823.5), (23.0, 33.1), (1.0, 40.0), (0.4, 0.66), (7.5, 1127.1), (93.8, 342.9), (16.8, 130.0), (280.7, 519.5), (1800.0, 6749.2307692307695), (5.483684210526316, 16.583478260869562), (23.298217179902757, 605.994944023113), (4.33, 7.17), (79.22, 161.34), (1369.72, 2473.9), (622.0, 1201.0), (532.0, 845.0), (372.0, 867.0), (219.0, 315.0)]
ni_in_min_norm -3.610526315789473
fe_in_min_norm -2.2116935483870974
sio2_in_min_norm -4.917134831460671
cao_in_min_norm -0.5631067961165047
mgo_in_min_norm -2.5992932862190816
al2o3_in_min_norm 0.0
fe_ni_min_norm -2.031311228334589
s_m_min_norm -2.361692044301347
bc_min_norm -2.4909037098345697
loi_in_min_norm -1.4026354319180088
mc_kilnf

In [20]:
constraints = [
        {'type': 'ineq', 'fun': lambda x: x[0] - ni_in_min_norm},  # ni_in > 0
        {'type': 'ineq', 'fun': lambda x: x[1] - fe_in_min_norm},  # fe_in > 0
        {'type': 'ineq', 'fun': lambda x: x[2] - sio2_in_min_norm},  # sio2_in > 0
        {'type': 'ineq', 'fun': lambda x: x[3] - cao_in_min_norm},  # cao_in > 0
        {'type': 'ineq', 'fun': lambda x: x[4] - mgo_in_min_norm},  # mgo_in > 0
        {'type': 'ineq', 'fun': lambda x: x[5] - al2o3_in_min_norm},  # al2o3_in > 0
        {'type': 'ineq', 'fun': lambda x: x[6] - fe_ni_min_norm},  # fe_ni > 0
        {'type': 'ineq', 'fun': lambda x: x[7] - s_m_min_norm},  # s_m > 0
        {'type': 'ineq', 'fun': lambda x: x[8] - bc_min_norm},  # bc > 0
        {'type': 'ineq', 'fun': lambda x: x[9] - loi_in_min_norm},  # loi_in > 0
        
        {'type': 'eq', 'fun': lambda x: mc_kilnfeed_exact_norm - x[10]},  # mc_kilnfeed tidak berubah
        {'type': 'eq', 'fun': lambda x: fc_coal_exact_norm - x[11]},  # fc_coal tidak berubah
        {'type': 'eq', 'fun': lambda x: gvc_coal_exact_norm - x[12]},  # gvc_coal tidak berubah
        {'type': 'eq', 'fun': lambda x: tco_exact_norm - x[13]},  # tco tidak berubah 

        {'type': 'ineq', 'fun': lambda x: x[14] - voltage_min_norm},  # voltage > 0
        {'type': 'ineq', 'fun': lambda x: x[15] - current_min_norm},  # current > 0
        {'type': 'ineq', 'fun': lambda x: x[16] - load_min_norm},  # load > 0
        {'type': 'ineq', 'fun': lambda x: x[17] - rpm_min_norm},  # rpm > 0
        {'type': 'ineq', 'fun': lambda x: x[18] - pry_p_min_norm},  # pry_p > 0
        {'type': 'ineq', 'fun': lambda x: x[19] - sec_p_min_norm},  # sec_p > 0
        {'type': 'ineq', 'fun': lambda x: x[20] - pry_v_min_norm},  # pry_v > 0
        {'type': 'ineq', 'fun': lambda x: x[21] - sec_v_min_norm},  # sec_v > 0
        {'type': 'ineq', 'fun': lambda x: x[22] - total_coal_min_norm},  # total_coal > 0
        {'type': 'ineq', 'fun': lambda x: x[23] - a_f_ratio_min_norm},  # a_f_ratio > 0

        {'type': 'eq', 'fun': lambda x: kg_tco_exact_norm - x[24]},  # kg_tco tidak berubah
    
        {'type': 'ineq', 'fun': lambda x: x[25] - reductor_ratio_min_norm},  # reductor_ratio > 0
        {'type': 'ineq', 'fun': lambda x: x[26] - reductor_consume_min_norm},  # reductor_consume > 0

        {'type': 'eq', 'fun': lambda x: charge_kiln_exact_norm - x[27]},  # charge_kiln tidak berubah

        {'type': 'ineq', 'fun': lambda x: x[28] - t_tic162_min_norm},  # t_tic162 > 556
        {'type': 'ineq', 'fun': lambda x: t_tic162_max_norm - x[28]},  # t_tic162 < 1201
        {'type': 'ineq', 'fun': lambda x: x[29] - t_tic163_min_norm},  # t_tic163 > 456
        {'type': 'ineq', 'fun': lambda x: t_tic163_max_norm - x[29]},  # t_tic163 < 845
        {'type': 'ineq', 'fun': lambda x: x[30] - t_tic166_min_norm},  # t_tic166 > 372
        {'type': 'ineq', 'fun': lambda x: t_tic166_max_norm - x[30]},  # t_tic166 < 867
        {'type': 'ineq', 'fun': lambda x: x[31] - t_tic172_min_norm},  # t_tic172 > 210
        {'type': 'ineq', 'fun': lambda x: t_tic172_max_norm - x[31]},  # t_tic172 < 315

    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][0] - T_furnace_norm},  # T_furnace > 1300
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][1] - ni_met_norm},    # Ni_met > 17
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][2] - C_met_low_norm},  # C_met > 0
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][3] - Si_met_low_norm},  # Si_met > 1
    {'type': 'ineq', 'fun': lambda x: Si_met_high_norm - ridge_model.predict(x.reshape(1, -1))[0][3]},  # Si_met < 2
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][4] - fe_met_low_norm},  # Fe_met > 0
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][5] - s_met_low_norm},  # S_met > 0
    {'type': 'ineq', 'fun': lambda x: s_met_high_norm - ridge_model.predict(x.reshape(1, -1))[0][5]},  # S_met < 0.4
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][6] - ni_slag_low_norm},  # Ni_slag > 0
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][7] - fe_slag_low_norm},  # Fe_slag > 0
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][8] -  T_kalsin_low_norm},    # T_kalsin > 600
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][9] - pic_161_low_norm},  # pic_161 > -3
    {'type': 'ineq', 'fun': lambda x: loi_kalsin_high - ridge_model.predict(x.reshape(1, -1))[0][10]},  # loi_kalsin < 1
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][10] - loi_kalsin_low },  # loi_kalsin > 0
]

# Jalankan optimisasi dengan constraints
result = minimize(objective_function, x0, method='SLSQP', constraints=constraints)

optimized_x = result.x
optimized_y = ridge_model.predict(optimized_x.reshape(1,-1))[0]

print("Optimized x:", optimized_x)
print("Optimized y:", optimized_y)

Optimized x: [ 0.57662229  0.37208151  0.63703922  0.22096626  0.9744287   0.52052476
  0.48029303  0.12983089  0.2525903   0.61599024  0.78096118  0.71162791
  0.          0.32122332  0.79270234  0.7845651   0.13873424  0.6191676
  1.05496442  0.67909321  0.98961322  1.03407554  0.73377644 -0.12056499
  0.05153598  1.0081527   0.65410404  0.79019725  0.77714387  0.63232323
  1.          0.51090715]
Optimized y: [ 0.51781745  1.12134489 -0.00461284  0.04637225  0.01023112 -0.07303877
  0.7642781   0.90936615  0.91873368  0.63013699  1.        ]


In [21]:
a=pd.DataFrame(optimized_x).T
a.columns=input_cols
a=pd.DataFrame(scaler_x.inverse_transform(a), columns=input_cols)
print(a.shape)
b=pd.DataFrame(optimized_y).T
b=pd.DataFrame(scaler_y.inverse_transform(b),columns=output_cols)
print(b.shape)

(1, 32)
(1, 11)


In [22]:
a

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,total_coal,a_f_ratio,kg_tco,reductor_ratio,reductor_consume,charge_kiln,t_tic162,t_tic163,t_tic166,t_tic172
0,1.988896,12.815524,39.545719,0.403798,25.284083,1.634448,6.800447,1.640635,0.514521,13.787213,...,5431.628949,4.145438,53.328066,7.193154,132.935023,2242.24,1071.9663,729.917172,867.0,268.047086


In [23]:
X_test_inverted.head()

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,total_coal,a_f_ratio,kg_tco,reductor_ratio,reductor_consume,charge_kiln,t_tic162,t_tic163,t_tic166,t_tic172
0,2.055,13.265,39.840,0.460,23.195,1.800,6.451669,1.718814,0.590599,13.195,...,5400.0,7.091111,53.328066,6.43,144.21,2242.24,964.000000,765.000000,584.000000,290.000000
1,2.070,15.505,37.615,0.510,20.935,2.930,7.491014,1.800052,0.566749,10.155,...,5000.0,7.420800,49.324258,6.37,134.23,2358.93,946.000000,815.000000,570.000000,276.000000
2,2.070,12.310,42.130,0.445,22.910,1.505,5.949930,1.838900,0.551500,11.580,...,5200.0,7.382308,57.784198,6.13,136.10,2221.20,963.000000,812.000000,503.000000,285.000000
3,2.055,15.165,37.190,0.560,21.310,0.000,7.378908,1.746586,0.583696,10.305,...,4700.0,7.382553,43.418014,6.67,150.41,2254.15,919.000000,756.000000,550.000000,260.000000
4,1.960,15.410,35.165,0.570,19.755,3.005,7.861458,1.780050,0.573375,14.105,...,4100.0,7.925854,47.012957,7.17,161.34,2249.42,906.071429,765.571429,534.785714,267.642857


In [24]:
b

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,1377.003011,22.976247,1.02791,1.008879,73.429192,0.4,0.066814,9.704154,732.708224,-1.0,1.04


In [25]:
y_test_inverted = scaler_y.inverse_transform(y_test)
y_test_inverted = pd.DataFrame(y_test_inverted, columns=output_cols)
y_test_inverted.head()

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,1377.000000,21.445000,1.030000,1.125000,74.661500,0.528500,0.064000,10.174000,640.000000,-4.000000,0.00
1,1374.500000,21.173333,1.188333,1.158333,74.656833,0.614833,0.066667,9.256667,660.000000,-2.700000,0.68
2,1346.400000,21.084000,1.326000,1.172000,74.524200,0.569800,0.070000,8.080000,611.000000,-3.800000,0.54
3,1386.666667,21.540000,1.130000,1.086667,74.542000,0.508667,0.064000,10.108000,592.000000,-3.100000,0.00
4,1373.000000,21.647500,1.065000,0.972500,74.673000,0.486000,0.062500,10.165000,635.285714,-3.357143,0.37


In [26]:
y_pred = ridge_model.predict(X_test.iloc[3, :].to_numpy().flatten().reshape(1, -1)).flatten()
y_pred[1]

# target_value = y_test.iloc[i, :].to_numpy()
# target_value[1]

0.7609548295336989

In [27]:
y_pred = ridge_model.predict(X_test.iloc[116, :].to_numpy().reshape(1, -1))
y_pred.shape

(1, 11)

In [28]:
target_value = y_test.iloc[0, :].to_numpy().flatten()
target_value.shape

(11,)

In [29]:
# Jumlah sampel yang ingin dioptimalkan
num_samples = len(X_test_inverted)

# Inisialisasi list untuk menyimpan hasil optimisasi
optimized_x_list = []
optimized_y_list = []

# Melakukan iterasi melalui semua sampel di X_test
for i in range(num_samples):
    print(f"\nMemproses Sampel ke-{i+1}")
    # Mengambil sampel ke-i dan meratakan array-nya
    x0 = X_test.iloc[i, :].to_numpy().flatten()

    # Pastikan y_test hanya memiliki satu kolom output jika prediksi satu fitur
    target_value = y_test.iloc[i, :].to_numpy()

    # Mendefinisikan fungsi objektif spesifik untuk sampel ke-i
    def objective_function(x_scaled):
        # Mengubah bentuk input untuk prediksi
        x_scaled_reshaped = x_scaled.reshape(1, -1)
        # Melakukan prediksi menggunakan model ridge
        y_pred = ridge_model.predict(x_scaled_reshaped)[0]
        # Menghitung selisih kuadrat (MSE)
        return np.sum(y_pred - target_value) ** 2

    mc_kilnfeed_exact_norm = x0[10]
    fc_coal_exact_norm = x0[11]
    gvc_coal_exact_norm = x0[12]
    tco_exact_norm = x0[13]
    kg_tco_exact_norm = x0[24]

    ni_in_exact_norm = x0[0]
    fe_in_exact_norm = x0[1]
    sio2_in_exact_norm = x0[2]
    cao_in_exact_norm = x0[3]
    mgo_in_exact_norm = x0[4]
    al2o3_in_exact_norm = x0[5]
    fe_ni_exact_norm = x0[6]
    s_m_exact_norm = x0[7]
    bc_exact_norm = x0[8]
    loi_in_exact_norm = x0[9]
    charge_kiln_exact_norm = x0[27]

    # Mendefinisikan constraints spesifik untuk sampel ke-i
    constraints_sample = [
        # Constraints equality (tidak berubah) (harusnya equality constraint, dikunci berdasarkan data asli)
        {'type': 'eq', 'fun': lambda x, ni_in_exact_norm=ni_in_exact_norm: x[0] - ni_in_exact_norm},  
        {'type': 'eq', 'fun': lambda x, fe_in_exact_norm=fe_in_exact_norm: x[1] - fe_in_exact_norm},
        {'type': 'eq', 'fun': lambda x, sio2_in_exact_norm=sio2_in_exact_norm: x[2] - sio2_in_exact_norm},  
        {'type': 'eq', 'fun': lambda x, cao_in_exact_norm=cao_in_exact_norm: x[3] - cao_in_exact_norm}, 
        {'type': 'eq', 'fun': lambda x, mgo_in_exact_norm=mgo_in_exact_norm: x[4] - mgo_in_exact_norm},  
        {'type': 'eq', 'fun': lambda x, al2o3_in_exact_norm=al2o3_in_exact_norm: x[5] - al2o3_in_exact_norm}, 
        {'type': 'eq', 'fun': lambda x, fe_ni_exact_norm=fe_ni_exact_norm: x[6] - fe_ni_exact_norm},
        {'type': 'eq', 'fun': lambda x, s_m_exact_norm=s_m_exact_norm: x[7] - s_m_exact_norm}, 
        {'type': 'eq', 'fun': lambda x, bc_exact_norm=bc_exact_norm: x[8] - bc_exact_norm}, 
        {'type': 'eq', 'fun': lambda x, loi_in_exact_norm=loi_in_exact_norm: x[9] - loi_in_exact_norm},

        # Constraints equality (tidak berubah)
        {'type': 'eq', 'fun': lambda x, mc_kilnfeed_exact_norm=mc_kilnfeed_exact_norm: mc_kilnfeed_exact_norm - x[10]},  # mc_kilnfeed tidak berubah
        {'type': 'eq', 'fun': lambda x, fc_coal_exact_norm=fc_coal_exact_norm: fc_coal_exact_norm - x[11]},  # fc_coal tidak berubah
        {'type': 'eq', 'fun': lambda x, gvc_coal_exact_norm=gvc_coal_exact_norm: gvc_coal_exact_norm - x[12]},  # gvc_coal tidak berubah
        {'type': 'eq', 'fun': lambda x, tco_exact_norm=tco_exact_norm: tco_exact_norm - x[13]},  # tco tidak berubah

        # Constraints input >= min_norm
        {'type': 'ineq', 'fun': lambda x, voltage_min_norm=voltage_min_norm: x[14] - voltage_min_norm},  # voltage > voltage_min_norm
        {'type': 'ineq', 'fun': lambda x, current_min_norm=current_min_norm: x[15] - current_min_norm},  # current > current_min_norm
        {'type': 'ineq', 'fun': lambda x, load_min_norm=load_min_norm: x[16] - load_min_norm},  # load > load_min_norm
        {'type': 'ineq', 'fun': lambda x, rpm_min_norm=rpm_min_norm: x[17] - rpm_min_norm},  # rpm > rpm_min_norm
        {'type': 'ineq', 'fun': lambda x, pry_p_min_norm=pry_p_min_norm: x[18] - pry_p_min_norm},  # pry_p > pry_p_min_norm
        {'type': 'ineq', 'fun': lambda x, sec_p_min_norm=sec_p_min_norm: x[19] - sec_p_min_norm},  # sec_p > sec_p_min_norm
        {'type': 'ineq', 'fun': lambda x, pry_v_min_norm=pry_v_min_norm: x[20] - pry_v_min_norm},  # pry_v > pry_v_min_norm
        {'type': 'ineq', 'fun': lambda x, sec_v_min_norm=sec_v_min_norm: x[21] - sec_v_min_norm},  # sec_v > sec_v_min_norm
        {'type': 'ineq', 'fun': lambda x, total_coal_min_norm=total_coal_min_norm: x[22] - total_coal_min_norm},  # total_coal > total_coal_min_norm
        {'type': 'ineq', 'fun': lambda x, a_f_ratio_min_norm=a_f_ratio_min_norm: x[23] - a_f_ratio_min_norm},  # a_f_ratio > a_f_ratio_min_norm

        # Constraints equality (tidak berubah)
        {'type': 'eq', 'fun': lambda x, kg_tco_exact_norm=kg_tco_exact_norm: kg_tco_exact_norm - x[24]},  # kg_tco tidak berubah

        # Constraints input >= min_norm
        {'type': 'ineq', 'fun': lambda x, reductor_ratio_min_norm=reductor_ratio_min_norm: x[25] - reductor_ratio_min_norm},  # reductor_ratio > reductor_ratio_min_norm
        {'type': 'ineq', 'fun': lambda x, reductor_consume_min_norm=reductor_consume_min_norm: x[26] - reductor_consume_min_norm},  # reductor_consume > reductor_consume_min_norm

        # Constraints equality (tidak berubah)
        {'type': 'eq', 'fun': lambda x, charge_kiln_exact_norm=charge_kiln_exact_norm: charge_kiln_exact_norm - x[27]},  # charge_kiln tidak berubah

        # Constraints input >= min_norm dan <= max_norm
        {'type': 'ineq', 'fun': lambda x, t_tic162_min_norm=t_tic162_min_norm: x[28] - t_tic162_min_norm},  # t_tic162 > t_tic162_min_norm
        {'type': 'ineq', 'fun': lambda x, t_tic162_max_norm=t_tic162_max_norm: t_tic162_max_norm - x[28]},  # t_tic162 < t_tic162_max_norm
        {'type': 'ineq', 'fun': lambda x, t_tic163_min_norm=t_tic163_min_norm: x[29] - t_tic163_min_norm},  # t_tic163 > t_tic163_min_norm
        {'type': 'ineq', 'fun': lambda x, t_tic163_max_norm=t_tic163_max_norm: t_tic163_max_norm - x[29]},  # t_tic163 < t_tic163_max_norm
        {'type': 'ineq', 'fun': lambda x, t_tic166_min_norm=t_tic166_min_norm: x[30] - t_tic166_min_norm},  # t_tic166 > t_tic166_min_norm
        {'type': 'ineq', 'fun': lambda x, t_tic166_max_norm=t_tic166_max_norm: t_tic166_max_norm - x[30]},  # t_tic166 < t_tic166_max_norm
        {'type': 'ineq', 'fun': lambda x, t_tic172_min_norm=t_tic172_min_norm: x[31] - t_tic172_min_norm},  # t_tic172 > t_tic172_min_norm
        {'type': 'ineq', 'fun': lambda x, t_tic172_max_norm=t_tic172_max_norm: t_tic172_max_norm - x[31]},  # t_tic172 < t_tic172_max_norm

        # Constraints berdasarkan prediksi model Ridge
        {'type': 'ineq', 'fun': lambda x, T_furnace_norm=T_furnace_norm: ridge_model.predict(x.reshape(1, -1))[0][0] - T_furnace_norm},  # T_furnace > T_furnace_norm
        {'type': 'ineq', 'fun': lambda x, ni_met_norm=ni_met_norm: ridge_model.predict(x.reshape(1, -1))[0][1] - ni_met_norm},    # Ni_met > ni_met_norm
        {'type': 'ineq', 'fun': lambda x, C_met_low_norm=C_met_low_norm: ridge_model.predict(x.reshape(1, -1))[0][2] - C_met_low_norm},  # C_met > C_met_low_norm
        {'type': 'ineq', 'fun': lambda x, Si_met_low_norm=Si_met_low_norm: ridge_model.predict(x.reshape(1, -1))[0][3] - Si_met_low_norm},  # Si_met > Si_met_low_norm
        {'type': 'ineq', 'fun': lambda x, Si_met_high_norm=Si_met_high_norm: Si_met_high_norm - ridge_model.predict(x.reshape(1, -1))[0][3]},  # Si_met < Si_met_high_norm
        {'type': 'ineq', 'fun': lambda x, fe_met_low_norm=fe_met_low_norm: ridge_model.predict(x.reshape(1, -1))[0][4] - fe_met_low_norm},  # Fe_met > fe_met_low_norm
        {'type': 'ineq', 'fun': lambda x, s_met_low_norm=s_met_low_norm: ridge_model.predict(x.reshape(1, -1))[0][5] - s_met_low_norm},  # S_met > s_met_low_norm
        {'type': 'ineq', 'fun': lambda x, s_met_high_norm=s_met_high_norm: s_met_high_norm - ridge_model.predict(x.reshape(1, -1))[0][5]},  # S_met < s_met_high_norm
        {'type': 'ineq', 'fun': lambda x, ni_slag_low_norm=ni_slag_low_norm: ridge_model.predict(x.reshape(1, -1))[0][6] - ni_slag_low_norm},  # Ni_slag > ni_slag_low_norm
        {'type': 'ineq', 'fun': lambda x, fe_slag_low_norm=fe_slag_low_norm: ridge_model.predict(x.reshape(1, -1))[0][7] - fe_slag_low_norm},  # Fe_slag > fe_slag_low_norm
        {'type': 'ineq', 'fun': lambda x, T_kalsin_low_norm=T_kalsin_low_norm: ridge_model.predict(x.reshape(1, -1))[0][8] - T_kalsin_low_norm},    # T_kalsin > T_kalsin_low_norm
        {'type': 'ineq', 'fun': lambda x, pic_161_low_norm=pic_161_low_norm: ridge_model.predict(x.reshape(1, -1))[0][9] - pic_161_low_norm},  # pic_161 > pic_161_low_norm
        {'type': 'ineq', 'fun': lambda x, loi_kalsin_high=loi_kalsin_high: loi_kalsin_high - ridge_model.predict(x.reshape(1, -1))[0][10]},  # loi_kalsin < loi_kalsin_high
        {'type': 'ineq', 'fun': lambda x, loi_kalsin_low=loi_kalsin_low: ridge_model.predict(x.reshape(1, -1))[0][10] - loi_kalsin_low },  # loi_kalsin > loi_kalsin_low
    ]

    # Melakukan optimisasi menggunakan metode SLSQP dengan constraints yang didefinisikan
    result = minimize(
        objective_function,
        x0,
        method='SLSQP',
        constraints=constraints_sample
    )

    # Memeriksa apakah optimisasi berhasil
    if result.success:
        optimized_x = result.x
        optimized_y = ridge_model.predict(optimized_x.reshape(1, -1))[0]

        # Menyimpan input dan output yang dioptimalkan
        optimized_x_list.append(optimized_x)
        optimized_y_list.append(optimized_y)
    else:
        print(f"Optimisasi gagal untuk sampel {i+1}: {result.message}")
        # Menambahkan NaN jika optimisasi gagal
        optimized_x_list.append([np.nan] * len(input_cols))
        optimized_y_list.append([np.nan] * len(output_cols))


# Mengubah list input yang dioptimalkan menjadi DataFrame
optimized_x_df = pd.DataFrame(optimized_x_list, columns=input_cols)

# Inverse transform input yang dioptimalkan ke skala asli
optimized_x_df_original = pd.DataFrame(
    scaler_x.inverse_transform(optimized_x_df.fillna(0)),
    columns=input_cols
)

# Mengubah list output yang dioptimalkan menjadi DataFrame
optimized_y_df = pd.DataFrame(optimized_y_list, columns=output_cols)

# Inverse transform output yang dioptimalkan ke skala asli
optimized_y_df_original = pd.DataFrame(
    scaler_y.inverse_transform(optimized_y_df.fillna(0)),
    columns=output_cols
)

# Menampilkan hasil optimisasi untuk setiap sampel
for i in range(num_samples):
    print(f"\nSampel {i+1}:")
    if not optimized_x_df_original.iloc[i].isnull().any():
        print("Input yang Dioptimalkan:", optimized_x_df_original.iloc[i].values)
        print("Output yang Dioptimalkan:", optimized_y_df_original.iloc[i].values)
    else:
        print("Optimisasi tidak berhasil untuk sampel ini.")

# Fungsi untuk menghasilkan nama file unik dengan suffix numerik
def generate_unique_filename(base_name, extension):
    filename = f"{base_name}.{extension}"
    counter = 1
    while os.path.exists(filename):
        filename = f"{base_name}_{counter}.{extension}"
        counter += 1
    return filename

# Menentukan nama file dasar dan ekstensi
base_filename = 'hasil_optimisasi'
file_extension = 'xlsx'

# Menghasilkan nama file unik
unique_filename = generate_unique_filename(base_filename, file_extension)

# Mengekspor hasil yang dioptimalkan ke file Excel dengan nama unik
with pd.ExcelWriter(unique_filename) as writer:
    optimized_x_df_original.to_excel(writer, sheet_name='Input_Dioptimalkan', index=False)
    optimized_y_df_original.to_excel(writer, sheet_name='Output_Dioptimalkan', index=False)

print(f"\nHasil optimisasi telah diekspor ke file '{unique_filename}'.")



Memproses Sampel ke-1

Memproses Sampel ke-2

Memproses Sampel ke-3

Memproses Sampel ke-4

Memproses Sampel ke-5

Memproses Sampel ke-6

Memproses Sampel ke-7

Memproses Sampel ke-8

Memproses Sampel ke-9

Memproses Sampel ke-10

Memproses Sampel ke-11

Memproses Sampel ke-12

Memproses Sampel ke-13

Memproses Sampel ke-14

Memproses Sampel ke-15

Memproses Sampel ke-16

Memproses Sampel ke-17

Memproses Sampel ke-18

Memproses Sampel ke-19

Memproses Sampel ke-20

Memproses Sampel ke-21

Memproses Sampel ke-22

Memproses Sampel ke-23

Memproses Sampel ke-24

Memproses Sampel ke-25

Memproses Sampel ke-26

Memproses Sampel ke-27

Memproses Sampel ke-28

Memproses Sampel ke-29

Memproses Sampel ke-30

Memproses Sampel ke-31

Memproses Sampel ke-32

Memproses Sampel ke-33

Memproses Sampel ke-34

Memproses Sampel ke-35

Memproses Sampel ke-36

Memproses Sampel ke-37

Memproses Sampel ke-38

Memproses Sampel ke-39

Memproses Sampel ke-40

Memproses Sampel ke-41

Memproses Sampel ke-42

